# Load Data and Modules

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [3]:
from preprocessing_utils import pickle_save, pickle_load

In [4]:
from sklearn.model_selection import train_test_split
from scipy.spatial.transform import Rotation as R

In [5]:
ellipse_cols = ['sat_id', 'ellipse_id', 'major', 'minor', 'loss', 'r00', 'r01', 'r02', 'r10', 'r11', 'r12', 'r20', 'r21', 'r22',
                't1', 't2', 't3']

In [6]:
data_sim = pd.read_csv('../data/data_sim.csv')
data_real = pd.read_csv('../data/data_real.csv')

In [7]:
data_sim = pd.DataFrame(data_sim.values, columns = ellipse_cols)
data_real = pd.DataFrame(data_real.values, columns = ellipse_cols)

In [8]:
data_sim.shape, data_real.shape

((38629, 17), (26790, 17))

# Axis Angle Representation

In [9]:
def convert_matrix(data):
    matrix = data.loc[:, 'r00':'r22'].values.reshape(-1, 3, 3)
    rotvec = np.array([R.from_matrix(m).as_rotvec() for m in matrix])
    phis = np.linalg.norm(rotvec,axis=1,keepdims=True)
    rotvec = rotvec/phis
    data['r1'], data['r2'], data['r3'] = np.transpose(rotvec)
    data['phi'] = phis[:,0]
    print(rotvec.shape)
    data.drop(['r00', 'r01', 'r02', 'r10', 'r11', 'r12', 'r20', 'r21', 'r22'], axis=1, inplace=True)

In [10]:
convert_matrix(data_sim)
convert_matrix(data_real)

(38629, 3)
(26790, 3)


# Train-Test Segregation

In [11]:
merged_data = data_sim.merge(data_real, how='outer', on=['sat_id', 'ellipse_id'], suffixes=['_sim', '_real'], indicator=True)
merge = merged_data['_merge']
merged_data.drop(['ellipse_id', 'loss_sim', 'loss_real'], axis=1, inplace=True)

In [12]:
train_data = merged_data[merged_data['_merge'] == 'both'].drop('_merge', axis=1).reset_index(drop=True)
test_data = merged_data[merged_data['_merge'] == 'left_only'].drop('_merge', axis=1).dropna(axis=1).reset_index(drop=True)

In [13]:
sat_ids_train = train_data['sat_id'].astype(int)
sat_ids_test = test_data['sat_id'].astype(int)

In [14]:
train_data.drop('sat_id', axis=1, inplace=True)
test_data.drop('sat_id', axis=1, inplace=True)

In [15]:
labels = train_data.loc[:, 'major_real':]
train_data = train_data.loc[:, :'phi_sim']

In [16]:
cols = ['major', 'minor', 't1', 't2', 't3', 'r1', 'r2', 'r3','phi']

In [17]:
train_data = pd.DataFrame(train_data.values, columns=cols)
test_data = pd.DataFrame(test_data.values, columns=cols)
labels = pd.DataFrame(labels.values, columns=cols)

In [18]:
train_data.shape, test_data.shape, labels.shape

((26790, 9), (11839, 9), (26790, 9))

# Train-CV Split per Satellite

In [19]:
split_ratio = 0.1

In [20]:
train = {}
cv = {}
labels_train = {}
labels_cv = {}

In [21]:
for sat_id in range(600):
    sat_data = train_data[sat_ids_train == sat_id]
    sat_labels = labels[sat_ids_train == sat_id]
    train_size = int(sat_data.shape[0] * (1 - split_ratio))
    train[sat_id] = sat_data.iloc[:train_size, :].reset_index(drop=True)
    cv[sat_id] = sat_data.iloc[train_size:, :].reset_index(drop=True)
    labels_train[sat_id] = sat_labels.iloc[:train_size, :].reset_index(drop=True)
    labels_cv[sat_id] = sat_labels.iloc[train_size:, :].reset_index(drop=True)

In [22]:
def relative_error(sat_id,field):
    delta = np.mean(np.abs(labels_train[sat_id][field]))
    den = np.mean(np.abs(train[sat_id][field]))
    return delta/den

# Network Architecture

In [23]:
class SatNet(nn.Module):
    
    def __init__(self, input_size, hidden_size):
        super(SatNet, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.fc2 = nn.Linear(hidden_size, input_size)
    
    def forward(self, x):
        a1 = torch.relu(self.fc1(x))
        a2 = self.fc2(a1)
        return a2+x

In [24]:
def train_satnet(sat_data, satnet_params, iterations, lr,l2 = 1e-8):
    train, cv, labels_train, labels_cv = [torch.from_numpy(data).float() for data in sat_data]
    input_size, hidden_size = satnet_params
    model = SatNet(input_size, hidden_size)
    optim = torch.optim.Adam(model.parameters(), lr,weight_decay = l2)
    
    for itr in range(iterations):
        optim.zero_grad()
        pred = model(train)
        pred_cv = model(cv)

        axis_pred =  pred[:,:2]
        trans_pred = pred[:,2:5]
        rot_pred = pred[:,5:8]/(1e-5+torch.sqrt(torch.sum(pred[:,5:8]**2,dim=1,keepdim=True)))
        phi_pred = torch.fmod(pred[:,8],2*np.pi)

        axis_act =  labels_train[:,:2]
        trans_act = labels_train[:,2:5]
        rot_act = labels_train[:,5:8]/(1e-5+torch.sqrt(torch.sum(labels_train[:,5:8]**2,dim=1,keepdim=True)))
        phi_act = torch.fmod(labels_train[:,8],2*np.pi)

        loss = torch.mean((axis_pred-axis_act)**2) + torch.mean((trans_pred-trans_act)**2) + torch.mean((rot_pred-rot_act)**2) + torch.mean((phi_act-phi_pred)**2)
        

        loss.backward()
        optim.step()
        if(itr % 1000 == 0):
            
            axis_pred =  pred_cv[:,:2]
            trans_pred = pred_cv[:,2:5]
            rot_pred = pred_cv[:,5:8]/(1e-5+torch.sqrt(torch.sum(pred_cv[:,5:8]**2,dim=1,keepdim=True)))
            phi_pred = torch.fmod(pred_cv[:,8],2*np.pi)

            axis_act =  labels_cv[:,:2]
            trans_act = labels_cv[:,2:5]
            rot_act = labels_cv[:,5:8]/(1e-5+torch.sqrt(torch.sum(labels_cv[:,5:8]**2,dim=1,keepdim=True)))
            phi_act = torch.fmod(labels_cv[:,8],2*np.pi)

            axis_loss = torch.mean((axis_pred-axis_act)**2)
            trans_loss = torch.mean((trans_pred-trans_act)**2) 
            rot_loss = torch.mean((rot_pred-rot_act)**2) 
            phi_loss = torch.mean((phi_act-phi_pred)**2) 

            cv_loss = axis_loss + trans_loss + rot_loss + phi_loss

            print('Iteration: {} | Loss (Train): {} | Loss (CV): {}'.format(itr, loss.item(), cv_loss.item()))
            print('CV - axis {} trans {} rot {} phi {}'.format(axis_loss.item(),trans_loss.item(),rot_loss.item(),phi_loss.item()))
   
    return model

In [25]:
input_size = train_data.shape[1]
hidden_size = 60
iterations = 10000
lr = 0.0001
satnet_params = [input_size, hidden_size]

In [26]:
train_satnets = False

In [27]:
if (train_satnets):
    models = {}
    for sat_id in range(600):
        print('\nSatellite ID:', sat_id,'\n')
        sat_data = [train[sat_id].values, cv[sat_id].values, labels_train[sat_id].values, labels_cv[sat_id].values]
        models[sat_id] = train_satnet(sat_data, satnet_params, iterations, lr, l2=1e-5)
        pickle_save(models, '../data/satnets.pickle')
else:
    models = pickle_load('../data/satnets.pickle')

# Predictions

In [73]:
data_pred = np.array([])
data_sim_format = data_sim.drop(['sat_id', 'ellipse_id', 'loss'], axis=1)
for sat_id in range(600):
    sat_data = torch.from_numpy(data_sim_format[data_sim['sat_id'] == sat_id].values).float()
    sat_pred = models[sat_id](sat_data).view(-1).detach().numpy()
    data_pred = np.concatenate([data_pred, sat_pred])

In [74]:
data_pred = data_pred.reshape(-1, data_sim_format.shape[1])

In [89]:
data_pred = data_sim.loc[:, ['sat_id', 'ellipse_id']].astype(int).join(pd.DataFrame(data_pred, columns=test_data.columns))

In [97]:
# Axis Angle Format
for i in range(1,4):
    col = 'r' + str(i)
    data_pred[col] = data_pred[col] * data_pred['phi']

In [98]:
data_pred.drop('phi', axis=1, inplace=True)

In [100]:
data_pred.to_csv('../data/data_pred.csv', index=False)